In [1]:
import gensim.downloader as api
#dataset = api.load("text8")
import math
import numpy as np
import h5py
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  
import sys
sys.path.append("..")
import tensorflow as tf
from Vocabulary import *
import time
tf.keras.backend.clear_session()
import csv
import cloudpickle
from csv_writer import *

In [2]:
class lr_On_plato:
    lowest_loss = -1
    lowest_time = 0
    patience = 10
    factor = 0.005
    
    def notify_loss(self,loss,epoch):
        if(self.lowest_loss == -1):
            self.lowest_loss = loss
            self.lowest_time = epoch
        if(loss < self.lowest_loss):
            self.lowest_loss = loss
            self.lowest_time = epoch
        if(loss > self.lowest_loss and self.lowest_time + 10 < epoch):
            self.lowest_loss = loss
            self.lowest_time = epoch
            print("decreased LR")
            self.factor = self.factor * 0.5
    
    def get_lr(self,epoch):
        return self.factor
        

In [3]:
class ModelTrainer:
    def __init__(self,vocab_length,block_path,vector_size = 300):
        self.vector_size = vector_size
        self.tf_con_bias = None
        self.tf_bias = None
        self.weights = None
        self.tf_weights = None
        self.zeilen = 0
        self.spalten = 0
        # AND HERE IT IS AGAIN
        self.block_length = 20000
        self.amount_split = math.ceil(vocab_length/self.block_length)
        self.block_path = block_path
        self.vocab_length = vocab_length
        self.ones_symetrical = tf.ones((self.block_length,self.block_length), dtype=tf.dtypes.float32, name=None)
    
    def prepare(self,filename):
        self.f = h5py.File('S:\\{filename}.hdf5'.format(filename=filename), "w")#plus experiment name
        #initalize all the HDF files
        self.con_weights = self.f.create_dataset("context-weights", (self.vocab_length, self.vector_size))
        self.weights = self.f.create_dataset("weights",(self.vector_size,self.vocab_length))
        self.context_bias = self.f.create_dataset("context-bias", (self.vocab_length,1))
        self.bias = self.f.create_dataset("bias", (1,self.vocab_length))

        self.init_matrices()
    
    def resume(self,filename):
        self.f = h5py.File('S:\\{filename}.hdf5'.format(filename=filename), "r+")#plus experiment name
        #initalize all the HDF files
        self.con_weights = self.f.get("context-weights")
        self.weights = self.f.get("weights")
        self.context_bias = self.f.get("context-bias")
        self.bias = self.f.get("bias")

    
    def init_matrices(self,chunk_size=10000):
        self.init_hdf_matrix(self.weights,-0.5,0.5,chunk_size)
        self.init_hdf_matrix(self.con_weights,-0.5,0.5,chunk_size)
        self.init_hdf_matrix(self.context_bias,-0.5,0.5,chunk_size)
        self.init_hdf_matrix(self.bias,-0.5,0.5,chunk_size)
    
    def init_hdf_matrix(self,hdf_data,min_value,max_value,block_length):
        if len(hdf_data) > len(hdf_data[0]):
            iterations = int(math.ceil(len(hdf_data) / block_length))
            for i in range(iterations):
                current_size = min(block_length,len(hdf_data)-block_length*i)
                hdf_data[i*block_length:(i+1)*block_length , :] = np.random.rand(current_size,len(hdf_data[0]))/self.vector_size
        else:
            iterations = int(math.ceil(len(hdf_data[0]) / block_length))
            for i in range(iterations):
                current_size = min(block_length,len(hdf_data[0])-block_length*i)
                hdf_data[:,i*block_length:(i+1)*block_length] = np.random.rand(len(hdf_data),current_size)/self.vector_size
            
    
    def load_blocks(self,zeilen,spalten):
        # load the hdf coocurence block
        if(zeilen >= spalten):
            template = "co_occurence_{i}_{j}.hdf5".format(i=zeilen,j=spalten)
        else:
            template = "co_occurence_{i}_{j}.hdf5".format(i=spalten,j=zeilen)
        file_path =  self.block_path + '\\' + template
        
        tmp_hf = h5py.File(file_path, "r+")
        coocurrence = tmp_hf.get("co-ocurrence")[:]
        if (spalten > zeilen):
            coocurrence = np.transpose(coocurrence)
        self.tf_co_occurences = tf.convert_to_tensor(coocurrence,dtype=tf.dtypes.float32)
        coocurrence = None
        tmp_hf.close()
        #Use normal matrix, if epsilon Shift, than add one to co-ocurence table to fix scaling and log
        self.tf_con_bias = tf.Variable(initial_value=self.context_bias[zeilen*self.block_length:(zeilen+1)*self.block_length,:],dtype=tf.dtypes.float32)
        self.tf_bias = tf.Variable(initial_value=self.bias[:,spalten*self.block_length:(spalten+1)*self.block_length],dtype=tf.dtypes.float32)
        self.tf_con_weights =  tf.Variable(initial_value=self.con_weights[zeilen*self.block_length:(zeilen+1)*self.block_length,:],dtype=tf.dtypes.float32)
        self.tf_weights = tf.Variable(initial_value=self.weights[:,spalten*self.block_length:(spalten+1)*self.block_length],dtype=tf.dtypes.float32)

    def reload_weights(self,zeilen,spalten):
        self.tf_con_bias = tf.Variable(initial_value=self.context_bias[zeilen*self.block_length:(zeilen+1)*self.block_length,:],dtype=tf.dtypes.float32)
        self.tf_bias = tf.Variable(initial_value=self.bias[:,spalten*self.block_length:(spalten+1)*self.block_length],dtype=tf.dtypes.float32)
        self.tf_con_weights =  tf.Variable(initial_value=self.con_weights[zeilen*self.block_length:(zeilen+1)*self.block_length,:],dtype=tf.dtypes.float32)
        self.tf_weights = tf.Variable(initial_value=self.weights[:,spalten*self.block_length:(spalten+1)*self.block_length],dtype=tf.dtypes.float32)
    
    def save_blocks(self,zeilen,spalten):
        self.context_bias[zeilen*self.block_length:(zeilen+1)*self.block_length,:] = self.tf_con_bias.numpy()
        self.bias[0,spalten*self.block_length:(spalten+1)*self.block_length] = self.tf_bias.numpy()
        self.con_weights[zeilen*self.block_length:(zeilen+1)*self.block_length,:] = self.tf_con_weights.numpy()
        self.weights[:,spalten*self.block_length:(spalten+1)*self.block_length] = self.tf_weights.numpy()
    
    def _close_files(self):
        self.f.close()
        

    def loss(self):    
        co_occurences = None
        #;ust the words context
        if(self.zeilen == self.amount_split - 1):
            difference = self.block_length - self.tf_con_bias.shape[0]
            add2_context_bias   = tf.zeros((difference,1),dtype=tf.dtypes.float32)
            add2_context_weights = tf.zeros((difference,self.vector_size),dtype=tf.dtypes.float32)
            
            context_weights       = tf.concat([self.tf_con_weights,add2_context_weights],axis = 0)
            bias_terms   = tf.concat([self.tf_con_bias,add2_context_bias],axis = 0) * self.ones_symetrical
        else:
            context_weights       = self.tf_con_weights
            bias_terms   = self.tf_con_bias * self.ones_symetrical
        
        co_occurences = self.tf_co_occurences
        #;ust the words without context
        if(self.spalten == self.amount_split - 1):
            difference = self.block_length - self.tf_bias.shape[1]
            add2_bias = tf.zeros((1,difference),dtype=tf.dtypes.float32)
            add2_weights = tf.zeros((self.vector_size,difference),dtype=tf.dtypes.float32)
            
            weights = tf.concat([self.tf_weights,add2_weights],axis = 1)
            bias_terms += tf.concat([self.tf_bias,add2_bias],axis=1) * self.ones_symetrical
        else:
            weights     = self.tf_weights
            bias_terms += self.tf_bias * self.ones_symetrical
          
        weight_matrix = tf.matmul(context_weights,weights)
        weight_matrix += bias_terms
        bias_terms = None
        weight_matrix -=  tf.math.log(tf.clip_by_value(co_occurences, clip_value_min = 1e-8,clip_value_max = 100000000000))
        weight_matrix = tf.math.square(weight_matrix)
        weight_matrix = self.cut_function2(co_occurences) * weight_matrix
        weight_matrix = tf.math.reduce_sum(weight_matrix)
        return weight_matrix
    
    alpha = tf.constant(0.75,dtype=tf.dtypes.float32)
    XMAX = tf.constant(100.0,dtype=tf.dtypes.float32)
    
    def cut_function2(self,value):
        clipped = tf.clip_by_value(value, clip_value_min = 0.0, clip_value_max=100.0)
        return tf.pow(clipped / self.XMAX, self.alpha)
    
    def load_optimizer(self,epoch,zeilen,spalten,optimizer_factory):
        #load optimizer & blocks
        if(epoch == 0):
            optimizer = optimizer_factory.create()
        else:
            name = 'S://optimizer{z}-{s}'.format(z = zeilen,s = spalten)
            with open(name, "rb") as file:
                optimizer = cloudpickle.load(file)
            optimizer.learning_rate.assign(lrOnPlato.get_lr(epoch))
        return optimizer
        
        
        
    def train_splitted(self,epochs,optimizer_factory,lrOnPlato,keepWeights=False):
        csv_writer = CSV_writer("hallo")
        #These can stay in memory
        
        for epoch in range(0,epochs):
            cur_loss = 0
            for zeilen in range(0,self.amount_split):
                for spalten in range(0,self.amount_split):
                    if spalten > zeilen:
                        continue
                    self.zeilen = zeilen
                    self.spalten = spalten
                    
                    optimizer = tf.keras.optimizers.Adam(0.001)
                    #train one side
                    self.load_blocks(zeilen,spalten)
                    #print(zeilen,spalten)
                    
                    #train code
                    with tf.GradientTape() as tape:
                        tmp_loss = self.loss()
                    grads = tape.gradient(tmp_loss, [self.tf_con_bias,self.tf_bias,self.tf_con_weights,self.tf_weights])
                    optimizer.apply_gradients(zip(grads, [self.tf_con_bias,self.tf_bias,self.tf_con_weights,self.tf_weights]))
                    cur_loss += tmp_loss
                    #print(optimizer.learning_rate)
                    self.save_blocks(zeilen,spalten)
                
                    
                    #train the other side
                    if spalten != zeilen:
                        tmp = self.zeilen
                        self.zeilen = self.spalten
                        self.spalten = tmp
                        
                        optimizer = tf.keras.optimizers.Adam(0.001)
                        self.reload_weights(self.zeilen,self.spalten)
                        self.tf_co_occurences = tf.transpose(self.tf_co_occurences)
                        #print(self.zeilen,self.spalten)
                    
                        #train code
                        with tf.GradientTape() as tape:
                            tmp_loss = self.loss()
                        grads = tape.gradient(tmp_loss, [self.tf_con_bias,self.tf_bias,self.tf_con_weights,self.tf_weights])
                        optimizer.apply_gradients(zip(grads, [self.tf_con_bias,self.tf_bias,self.tf_con_weights,self.tf_weights]))
                        cur_loss += tmp_loss
                        #print(optimizer.learning_rate)
                        self.save_blocks(self.zeilen,self.spalten)
                    
                        
                
            print('epoch'+str(epoch)+"loss:"+str(cur_loss.numpy()))
            lrOnPlato.notify_loss(cur_loss.numpy(),epoch)
            csv_writer.write(optimizer_factory.optimiser_name(),lrOnPlato.get_lr(epoch),epoch+1,cur_loss.numpy())
        self._close_files()
        return None

    

In [4]:
vocab = Vocabulary()
vocab.load('..\\vocabs\\base_vocabulary')
size = vocab.get_size()

class AdamFactory:
    def __init__(self,lr = 0.001):
        self.lr = lr
    def create(self):
        return tf.keras.optimizers.Adam(self.lr)
    def optimiser_name(self):
        return "Adam"
    
    
#tf.keras.backend.clear_session()
#trainer = ModelTrainer(size,"S:\\base_coocurrence_hdf")
#trainer.prepare("baseline_preSanityCheck")
#trainer.train_splitted(1,AdamFactory(),lr_On_plato())


import time
startTime = time.time()
trainer2 = ModelTrainer(size,'S:\\base_coocurrence_hdf')
trainer2.resume("baseline_preSanityCheck")

trainer2.train_splitted(1,AdamFactory(),lr_On_plato())

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

epoch0loss:13472589.0
Execution time in seconds: 616.5231535434723
